In [2]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("model_inference", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.profiler import profiler
import matplotlib.pyplot as plt
import time
from parse_dataset import NetworkDataset, parse_dataset, split_datasets, binary_dataset
from split_model import SplitModelDPU, SplitModelHost
from load_models import models
from transfer_tensors import HostSocket

In [4]:
conf = {
    "batch_size": 512,
    "epochs": 10,
    "learning_rate": 0.0001,
    "dpu": True
}

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
host_path = os.path.join(os.getcwd().replace("model_inference", ""), "checkpoint", "host_split_model.pth")
host_model = models["host"]
host_model.load(host_path)

Checkpoint loaded from /home/jorgetf/testmodel/Network-Packet-ML-Model/checkpoint/host_split_model.pth!


In [7]:
so_file = os.path.join(os.getcwd().replace("model_inference", ""), "socket_transfer", "socket_transfer.so")
socket = HostSocket(so_file)

In [8]:
logits = socket.receive()

In [10]:
print(logits[:10])
print(logits.shape)

tensor([[[ 0.7089, -0.7476, -0.0036,  ..., -1.1988, -0.0031,  0.0625]],

        [[ 0.7035, -0.8195, -0.6801,  ...,  1.1419, -0.9509,  1.0023]],

        [[ 0.7385, -0.6565, -0.3436,  ...,  0.9756, -0.9859,  1.0586]],

        ...,

        [[-1.4335,  1.3560,  1.2605,  ..., -1.1613, -0.4889,  0.3996]],

        [[-1.4335,  1.3560,  1.2605,  ...,  1.0344,  0.3211, -0.4307]],

        [[ 0.6609, -0.1933, -1.8079,  ...,  0.9212, -0.9846,  0.9950]]])
torch.Size([430, 1, 128])


In [11]:
targets = socket.receive()

In [13]:
targets = targets.to(dtype=torch.long)
print(targets[:10])
print(targets.shape)

tensor([ 0,  0, 14,  0, 12, 10, 15, 10, 19, 14])
torch.Size([430])


In [14]:
host_model.model.eval()
pred = host_model.inference(logits)

In [15]:
print(pred.shape)
print(pred[:10])
print(targets[:10])

torch.Size([430, 24])
tensor([[ 4.6979e+00,  3.1636e+00, -3.6517e+00, -3.6972e+00, -4.2605e+00,
         -4.6877e-01, -2.0419e+00, -4.6063e+00, -3.6925e+00, -3.7097e+00,
         -3.3261e+00, -1.8383e+00, -2.9790e+00, -3.6210e+00, -3.7775e+00,
          4.5890e+00,  2.2242e-02, -3.2947e+00, -1.7936e+00, -3.3861e+00,
         -3.5585e+00, -2.5873e+00, -6.3391e-01, -2.8784e+00],
        [ 6.0482e+00,  3.8152e+00, -3.1756e+00, -4.1994e+00, -5.5951e+00,
         -1.4789e+00, -5.4159e+00, -2.9915e+00, -1.7095e+00, -4.6667e+00,
         -6.5855e+00, -1.7636e+00, -2.1558e+00, -4.0477e+00, -4.9167e+00,
         -1.8678e+00,  2.1934e-01, -4.0542e+00, -2.5919e+00, -2.5418e+00,
         -5.2979e+00, -3.9423e+00, -6.8515e-01, -5.3517e+00],
        [-3.7170e+00, -6.4874e+00,  4.3721e+00,  1.5684e+00, -2.2964e+00,
         -1.5650e+00, -4.5363e+00, -5.7751e+00,  3.6249e+00, -4.1446e+00,
         -3.7369e+00, -2.8719e+00,  2.9706e+00,  2.8010e-01,  4.9406e+00,
         -5.6998e+00, -3.0746e+00, -2.73